<a href="https://colab.research.google.com/github/ssrbazpur/Envisioning-Yellow-Taxi-High-Demand-Areas-in-NYC-city/blob/master/Data_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3><center><b><U>Data Preparation</U></b></center> </h3>
<h4><center><b>Prepared By: Simranjeet Randhawa</b></center> </h4>
<h4><center><b>Student ID: 200412297</b></center> </h4>




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install datalab

In [0]:
from datalab.context import Context

In [0]:

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/drive/My Drive/Data Science/nyc-taxi-265120-baf9a3e4cf9b.json"
%reload_ext google.cloud.bigquery

In [0]:
project_id = 'nyc-taxi-265120' 
from google.cloud import bigquery
client = bigquery.Client(project = project_id)

In [0]:
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

<h3><b> 1. Data Preparation </b></h3>
<p align="justify"> Data preparation is termed as the process of cleaning the data so that the data can be transformed in a form that can be processed and analysed. It is an important step prior to processing and often involves reformatting data, making corrections to data and the combining of data sets to enrich data.
</p>

<h4><b> 1.1 Data Cleaning </b></h4>

<b><u> 2018 First Half Yellow Cab Data Cleaning (Big Query)</u></b>
<p align="justify"> 
<ol><li> Analyse if there is rows with total trip amount =0 ? </li>
<li>Is there rows with pick up time > drop off time.</li><li>  Analyse rows which have 0 passenger count.</li>
</ol>
</p>

In [0]:
# if numpy is not installed already : pip3 install numpy
import numpy as np#Do aritmetic operations on arrays

# matplotlib: used to plot graphs
import matplotlib
# matplotlib.use('nbagg') : matplotlib uses this protocall which makes plots more user intractive like zoom in and zoom out
matplotlib.use('nbagg')
import matplotlib.pylab as plt
import seaborn as sns#Plots
from matplotlib import rcParams#Size of plots 

<b> Finding the number of trips where amount given by passenger is less than zero.</b>

In [0]:
query = """
SELECT * FROM `nyc-taxi-265120.NYC.2018firsthalf` where total_amount < 0 
"""
df_head = client.query(query).to_dataframe()

In [24]:
df_head.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2018-01-01 19:39:42+00:00,2018-01-01 19:40:04+00:00,6,0.11,1,N,132,132,3,-2.5,-1.0,-0.5,0.0,0.0,-0.3,-4.3
1,2,2018-01-02 16:48:22+00:00,2018-01-02 16:48:45+00:00,2,0.04,1,N,148,148,4,-2.5,-1.0,-0.5,0.0,0.0,-0.3,-4.3
2,2,2018-01-02 17:07:45+00:00,2018-01-02 17:08:21+00:00,5,0.06,1,N,100,100,3,-2.5,-1.0,-0.5,0.0,0.0,-0.3,-4.3
3,2,2018-01-02 18:41:29+00:00,2018-01-02 18:42:04+00:00,6,0.08,1,N,170,170,4,-2.5,-1.0,-0.5,0.0,0.0,-0.3,-4.3
4,2,2018-01-02 19:45:28+00:00,2018-01-02 19:46:01+00:00,1,0.00,1,N,7,193,3,-2.5,-1.0,-0.5,0.0,0.0,-0.3,-4.3


In [0]:
query = """
SELECT count(*) FROM `nyc-taxi-265120.NYC.2018firsthalf` where total_amount < 0 
"""
df = client.query(query).to_dataframe()

In [23]:
print("Number of rows with amount of trip <0:")
print(df_head.shape[0])

Number of rows with amount of trip <0:
29860


<p align="justify"> There are 29860 rows in 2018 First half dataset where amount trip is less than 0 which is not possible as there is no online wallet system that NYC city taxi used in 2018 nor 2019. Hence this must be the possible outliers or there must be erroneous entry in the amount column. I decided to normalise the value and keep it.</p>

<b> Finding the number of pickups where pickup time is greater than drop off time</b>

In [0]:
query = """
SELECT * FROM `nyc-taxi-265120.NYC.2018firsthalf` where tpep_pickup_datetime > tpep_dropoff_datetime 
"""
df = client.query(query).to_dataframe()

In [26]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2018-01-01 15:15:13+00:00,2017-12-28 16:03:38+00:00,1,3.3,1,N,209,137,1,13.00,0.5,0.5,2.85,0.00,0.3,17.15
1,1,2018-02-04 15:15:17+00:00,2018-02-03 16:06:36+00:00,2,20.2,2,N,132,151,1,52.00,0.0,0.5,11.70,5.76,0.3,70.26
2,1,2018-02-11 15:15:01+00:00,2017-12-24 17:09:53+00:00,2,4.7,1,N,246,66,1,25.00,0.0,0.5,5.16,0.00,0.3,30.96
3,1,2018-02-17 23:32:17+00:00,2018-02-17 23:31:41+00:00,1,0.1,5,N,5,5,4,99.32,0.0,0.0,0.00,0.00,0.3,99.62
4,1,2018-02-09 09:50:19+00:00,2018-02-09 04:48:28+00:00,1,1.3,1,N,90,113,2,6.00,0.5,0.5,0.00,0.00,0.3,7.30


In [27]:
print("Number of rows where pickup time is greater than drop off:")
print(df.shape[0])

Number of rows where pickup time is greater than drop off:
18


<p align="justify"> I decided to remove these rows as they are certainty the outliers and will not make much difference</p>

<b> Finding the number of pickups where there are no passengers</b>

In [0]:
query = """
SELECT count(*) FROM `nyc-taxi-265120.NYC.2018firsthalf` where passenger_count = 0 
"""
df = client.query(query).to_dataframe()

In [38]:
print("Number of pickups where there are no passengers:")
print(df["f0_"][0])

Number of pickups where there are no passengers:
402117


In [0]:
query = """
SELECT  tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,PULocationID,DOLocationID,total_amount FROM `nyc-taxi-265120.NYC.2018firsthalf` LIMIT 10000
"""
df = client.query(query).to_dataframe()

Moving onto the Trip duration, according to NYC Taxi & Limousine Commision Regulations the maximum allowed trip duration in a 24 hour interval is 12 hours. So selecting only the data points whose trip times is greater than 1 and less than 720 minutes (12 hours). Given below is the PDF of log-trip-times along with its Normal Q-Q plot which says that the data follows a good normal distribution.<br>
<IMG SRC="https://github.com/ssrbazpur/Envisioning-Yellow-Taxi-High-Demand-Areas-in-NYC-city/blob/master/Screenshots/log_times.png?raw=true" heigh=400 width=400>

In [0]:
query = """
SELECT  * FROM `nyc-taxi-265120.NYC.2018firsthalf` WHERE TIMESTAMP_DIFF(tpep_dropoff_datetime,tpep_pickup_datetime,HOUR)>12 LIMIT 10000
"""
df = client.query(query).to_dataframe()

In [0]:
TIMESTAMP_DIFF(tpep_dropoff_datetime,tpep_pickup_datetime,SECOND)

In [70]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2018-01-16 07:11:32+00:00,2018-01-17 06:11:32+00:00,1,0.00,1,N,193,264,2,2.5,0.0,0.0,0.00,0.00,0.0,2.50
1,2,2018-01-14 22:20:12+00:00,2018-01-15 21:11:24+00:00,1,0.00,1,N,132,132,2,2.5,0.5,0.5,0.00,0.00,0.3,3.80
2,2,2018-01-15 00:32:46+00:00,2018-01-16 00:20:30+00:00,1,0.00,1,N,17,17,2,2.5,0.5,0.5,0.00,0.00,0.3,3.80
3,2,2018-01-14 21:49:02+00:00,2018-01-15 21:45:35+00:00,1,0.06,1,N,234,234,2,3.0,0.5,0.5,0.00,0.00,0.3,4.30
4,2,2018-01-15 05:02:20+00:00,2018-01-16 00:00:00+00:00,1,0.29,1,N,164,164,2,3.0,0.5,0.5,0.00,0.00,0.3,4.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2,2018-02-04 02:39:40+00:00,2018-02-05 02:32:13+00:00,2,8.00,1,N,244,230,2,34.0,0.5,0.5,0.00,0.00,0.3,35.30
9996,2,2018-02-04 01:07:12+00:00,2018-02-05 00:49:52+00:00,1,11.84,1,N,161,92,2,36.0,0.5,0.5,0.00,5.76,0.3,43.06
9997,2,2018-02-04 21:34:28+00:00,2018-02-05 21:07:02+00:00,1,12.26,1,N,164,89,2,36.0,0.5,0.5,0.00,0.00,0.3,37.30
9998,2,2018-02-04 17:42:09+00:00,2018-02-05 17:39:07+00:00,1,13.24,1,N,170,252,1,38.0,0.0,0.5,4.46,5.76,0.3,49.02


In [0]:
sns.boxplot(y="total_amount", data =df)
plt.show()

<b><u> 2018 Second Half Yellow Cab Data Cleaning (Big Query)</u></b>
<p align="justify"> 
<ol><li> Analyse if there is rows with total trip amount =0 ? </li>
<li>Is there rows with pick up time > drop off time.</li><li>  Analyse rows which have 0 passenger count.</li>
</ol>
</p>

<b> Finding the number of trips where amount given by passenger is less than zero.</b>

In [0]:
query = """
SELECT count(*) FROM `nyc-taxi-265120.NYC.2018SecondHalf` where total_amount <= 0 
"""
df = client.query(query).to_dataframe()

In [0]:
print("Number of rows with amount of trip <=0:")
print(df["f0_"][0])

<b> Finding the number of pickups where pickup time is greater than drop off time</b>

In [0]:
query = """
SELECT count(*) FROM `nyc-taxi-265120.NYC.2018SecondHalf` where tpep_pickup_datetime > tpep_dropoff_datetime 
"""
df = client.query(query).to_dataframe()

print("Number of rows where pickup time is greater than drop off:")
print(df["f0_"][0])

<b> Finding the number of pickups where there are no passengers</b>

In [0]:

query = """
SELECT count(*) FROM `nyc-taxi-265120.NYC.2018SecondHalf` where passenger_count = 0 
"""
df = client.query(query).to_dataframe()

print("Number of pickups where there are no passengers:")
print(df["f0_"][0])


In [0]:
query = """
SELECT * FROM `nyc-taxi-265120.NYC.2018SecondHalf`
"""
df = client.query(query).to_dataframe()

<b><u> 2019 First Half Yellow Cab Data Cleaning (Big Query)</u></b>
<p align="justify"> 
<ol><li> Analyse if there is rows with total trip amount =0 ? </li>
<li>Is there rows with pick up time > drop off time.</li><li>  Analyse rows which have 0 passenger count.</li>
</ol>
</p>

In [0]:
query = """
SELECT count(*) FROM `nyc-taxi-265120.NYC.2019firsthalf` where total_amount <= 0 LIMIT 10000
"""
df = client.query(query).to_dataframe()

In [0]:
print("Number of rows with amount of trip <=0:")
print(df["f0_"][0])

In [0]:
query = """
SELECT * FROM `nyc-taxi-265120.NYC.2019firsthalf` where tpep_pickup_datetime > tpep_dropoff_datetime LIMIT 10000
"""
df = client.query(query).to_dataframe()

print("Number of rows where pickup time is greater than drop off:")
print(df["f0_"][0])

In [0]:
query = """
SELECT count(*) FROM `nyc-taxi-265120.NYC.2019firsthalf` where passenger_count = 0 LIMIT 10000
"""
df = client.query(query).to_dataframe()
print("Number of pickups where there are no passengers:")
print(df["f0_"][0])

In [0]:
query = """
SELECT count(*) FROM `nyc-taxi-265120.NYC.2019firsthalf` where passenger_count = 0 and total_amount<=0 LIMIT 10000
"""
df = client.query(query).to_dataframe()

<b><u> 2019 Second Half Yellow Cab Data Cleaning (Big Query)</u></b>
<p align="justify"> 
<ol><li> Analyse if there is rows with total trip amount =0 ? </li>
<li>Is there rows with pick up time > drop off time.</li><li>  Analyse rows which have 0 passenger count.</li>
</ol>
</p>

In [0]:
query = """
SELECT count(*) FROM `nyc-taxi-265120.NYC.2019secondhalf` where total_amount <= 0 
"""
df = client.query(query).to_dataframe()
print("Number of rows with amount of trip <=0:")
print(df["f0_"][0])

In [0]:
query = """
SELECT * FROM `nyc-taxi-265120.NYC.2019secondhalf` where tpep_pickup_datetime > tpep_dropoff_datetime 
"""
df = client.query(query).to_dataframe()

print("Number of rows where pickup time is greater than drop off:")
print(df["f0_"][0])

In [0]:
query = """
SELECT count(*) FROM `nyc-taxi-265120.NYC.2019secondhalf` where passenger_count = 0 
"""
df = client.query(query).to_dataframe()
print("Number of pickups where there are no passengers:")
print(df["f0_"][0])

<b><u> INNER JOIN (Eliminating out of Bounds Location if exists)</u></b>

Eliminaing out of Bound Location with the help of INNER JOIN.<br>
<ol><li>
Table A- Taxi Trips Data </li>
<li>
Table B- Zone Data</li>
</ol>

<img src="https://helloacm.com/wp-content/uploads/2019/04/sql-joins-venn-diagrams-inner-join.png"/>

The INNER JOIN keyword selects all rows from both tables as long as there is a match between the columns. It gives only the trips pickups in the NYC region.</p>

In [0]:
#Sample showing how INNER JOIN can be done using Big Query.
query = """
with td as(
WITH AB AS
(
SELECT  CAST(tpep_pickup_datetime AS DATETIME) AS PICKUP,* FROM `nyc-taxi-265120.NYC.2019firsthalf` where total_amount>0
)  
SELECT  PICKUP,CAST(EXTRACT (YEAR from PICKUP) AS STRING) AS year,EXTRACT (DAYOFYEAR from PICKUP) AS daynumber,EXTRACT (HOUR from PICKUP) AS hour, cast(PULocationID as STRING) as LOCATION,COUNT(*) AS numtrips FROM AB group by PICKUP,year,daynumber,hour,LOCATION),points AS
(SELECT *,ST_CENTROID(zone_geom) as p FROM `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`)
SELECT cast(DATE(td.PICKUP) as STRING) AS A1,
points.zone_id,CAST(TIME(DATETIME_TRUNC(td.PICKUP, HOUR)) as string) rounded_to_hour,
SUM(td.numtrips) as label,
FROM  td 

INNER JOIN 

points ON points.zone_id=td.LOCATION AND
eXTRACT (YEAR from PICKUP) =2018 AND td.daynumber<150
group by zone_id,A1,rounded_to_hour
"""
df1 = client.query(query).to_dataframe()

<h4> <b>5.2 Data Wrangling </b></h4>
<p align="justify"> Data wrangling which is also referred as data munging.The main moto transform data into a format that is more appropriate and considered more valuable.</p>
<b><u> 2018 Data Wrangling</u></b>

<p align="justify">Add columns: year, month, weekday name, and hour using BIg Query</p>

In [0]:
query = """
with td as(
WITH AB AS
(
SELECT  CAST(tpep_pickup_datetime AS DATETIME) AS PICKUP,* FROM `nyc-taxi-265120.NYC.2018firsthalf` where total_amount>0 and  tpep_pickup_datetime < tpep_dropoff_datetime
)  
SELECT  PICKUP,CAST(EXTRACT (YEAR from PICKUP) AS STRING) AS year,EXTRACT (DAYOFYEAR from PICKUP) AS daynumber,EXTRACT (HOUR from PICKUP) AS hour, cast(PULocationID as STRING) as LOCATION,COUNT(*) AS numtrips FROM AB group by PICKUP,year,daynumber,hour,LOCATION),points AS
(SELECT *,ST_CENTROID(zone_geom) as p FROM `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`)
SELECT 
cast(DATE(td.PICKUP) as STRING) AS A1,
points.zone_id,
CAST(TIME(DATETIME_TRUNC(td.PICKUP, HOUR)) as string) rounded_to_hour,
SUM(td.numtrips) as label,
FROM  td INNER JOIN points ON points.zone_id=td.LOCATION AND
eXTRACT (YEAR from PICKUP) =2018 
group by zone_id,A1,rounded_to_hour
"""
df1 = client.query(query).to_dataframe()

In [0]:
import pandas as pd
df1["time"]=pd.to_datetime(df1['A1'] + ' ' + df1['rounded_to_hour'])
a=df1.pivot_table("label", "time", "zone_id")
b=df1.pivot_table("label", "time", "zone_id").unstack().reset_index()
b.rename( columns={0:'pickups'}, inplace=True )
b1=b.fillna(0)

In [0]:
b1.head()

,zone_id,time,pickups
0,1,2018-01-01 00:00:00,0.0
1,1,2018-01-01 01:00:00,0.0
2,1,2018-01-01 02:00:00,0.0
3,1,2018-01-01 03:00:00,1.0
4,1,2018-01-01 04:00:00,3.0


In [0]:
from datalab.context import Context
#Dont run this

b1.to_gbq('hello.2018DataFirsthalf', 
                 Context.default().project_id,
                 chunksize=500000, 
                 if_exists='append',
                 verbose=False
                 )

/usr/local/lib/python3.6/dist-packages/pandas_gbq/gbq.py:1127: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  stacklevel=1,
3it [00:56, 15.09s/it]


In [0]:
query = """
with td as(
WITH AB AS
(
SELECT  CAST(tpep_pickup_datetime AS DATETIME) AS PICKUP,* FROM `nyc-taxi-265120.NYC.2018SecondHalf`
)  
SELECT  PICKUP,CAST(EXTRACT (YEAR from PICKUP) AS STRING) AS year,EXTRACT (DAYOFYEAR from PICKUP) AS daynumber,EXTRACT (HOUR from PICKUP) AS hour, cast(PULocationID as STRING) as LOCATION,COUNT(*) AS numtrips FROM AB group by PICKUP,year,daynumber,hour,LOCATION),points AS
(SELECT *,ST_CENTROID(zone_geom) as p FROM `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`)
SELECT 
cast(DATE(td.PICKUP) as STRING) AS A1,
points.zone_id,
CAST(TIME(DATETIME_TRUNC(td.PICKUP, HOUR)) as string) rounded_to_hour,
SUM(td.numtrips) as label,
FROM  td INNER JOIN points ON points.zone_id=td.LOCATION AND
eXTRACT (YEAR from PICKUP) =2018 
group by zone_id,A1,rounded_to_hour
"""
df1 = client.query(query).to_dataframe()

In [0]:
import pandas as pd
df1["time"]=pd.to_datetime(df1['A1'] + ' ' + df1['rounded_to_hour'])
a=df1.pivot_table("label", "time", "zone_id")
b=df1.pivot_table("label", "time", "zone_id").unstack().reset_index()
b.rename( columns={0:'pickups'}, inplace=True )
b1=b.fillna(0)

In [0]:
from datalab.context import Context
#Dont run this

b1.to_gbq('hello.2018DataSecondhalf', 
                 Context.default().project_id,
                 chunksize=500000, 
                 if_exists='append',
                 verbose=False
                 )

/usr/local/lib/python3.6/dist-packages/pandas_gbq/gbq.py:1127: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  stacklevel=1,
3it [00:43, 11.34s/it]


<b><u> 2019 Data Wrangling</u></b>

In [0]:
query = """
with td as(
WITH AB AS
(
SELECT  CAST(tpep_pickup_datetime AS DATETIME) AS PICKUP,* FROM `nyc-taxi-265120.NYC.2019firsthalf`
)  
SELECT  PICKUP,CAST(EXTRACT (YEAR from PICKUP) AS STRING) AS year,EXTRACT (DAYOFYEAR from PICKUP) AS daynumber,EXTRACT (HOUR from PICKUP) AS hour, cast(PULocationID as STRING) as LOCATION,COUNT(*) AS numtrips FROM AB group by PICKUP,year,daynumber,hour,LOCATION),points AS
(SELECT *,ST_CENTROID(zone_geom) as p FROM `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`)
SELECT 
cast(DATE(td.PICKUP) as STRING) AS A1,
points.zone_id,
CAST(TIME(DATETIME_TRUNC(td.PICKUP, HOUR)) as string) rounded_to_hour,
SUM(td.numtrips) as label,
FROM  td INNER JOIN points ON points.zone_id=td.LOCATION AND
eXTRACT (YEAR from PICKUP) =2019 
group by zone_id,A1,rounded_to_hour
"""
df1 = client.query(query).to_dataframe()

In [0]:
import pandas as pd
df1["time"]=pd.to_datetime(df1['A1'] + ' ' + df1['rounded_to_hour'])
a=df1.pivot_table("label", "time", "zone_id")
b=df1.pivot_table("label", "time", "zone_id").unstack().reset_index()
b.rename( columns={0:'pickups'}, inplace=True )
b1=b.fillna(0)

In [0]:
from datalab.context import Context
#Dont run this

b1.to_gbq('hello.2019DataFirsthalf', 
                 Context.default().project_id,
                 chunksize=500000, 
                 if_exists='append',
                 verbose=False
                 )

/usr/local/lib/python3.6/dist-packages/pandas_gbq/gbq.py:1127: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  stacklevel=1,
3it [00:28,  8.09s/it]


In [0]:
query = """
with td as(
WITH AB AS
(
SELECT  CAST(tpep_pickup_datetime AS DATETIME) AS PICKUP,* FROM `nyc-taxi-265120.NYC.2019secondhalf`
)  
SELECT  PICKUP,CAST(EXTRACT (YEAR from PICKUP) AS STRING) AS year,EXTRACT (DAYOFYEAR from PICKUP) AS daynumber,EXTRACT (HOUR from PICKUP) AS hour, cast(PULocationID as STRING) as LOCATION,COUNT(*) AS numtrips FROM AB group by PICKUP,year,daynumber,hour,LOCATION),points AS
(SELECT *,ST_CENTROID(zone_geom) as p FROM `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`)
SELECT 
cast(DATE(td.PICKUP) as STRING) AS A1,
points.zone_id,
CAST(TIME(DATETIME_TRUNC(td.PICKUP, HOUR)) as string) rounded_to_hour,
SUM(td.numtrips) as label,
FROM  td INNER JOIN points ON points.zone_id=td.LOCATION AND
eXTRACT (YEAR from PICKUP) =2019 
group by zone_id,A1,rounded_to_hour
"""
df1 = client.query(query).to_dataframe()

In [0]:
df1.head()

,A1,zone_id,rounded_to_hour,label
0,2019-07-16,233,00:00:00,34
1,2019-10-31,87,00:00:00,64
2,2019-07-16,141,00:00:00,48
3,2019-07-17,90,00:00:00,94
4,2019-07-17,232,00:00:00,14


In [0]:
import pandas as pd
df1["time"]=pd.to_datetime(df1['A1'] + ' ' + df1['rounded_to_hour'])
a=df1.pivot_table("label", "time", "zone_id")
b=df1.pivot_table("label", "time", "zone_id").unstack().reset_index()
b.rename( columns={0:'pickups'}, inplace=True )
b1=b.fillna(0)

In [0]:
from datalab.context import Context
#Dont run this

b1.to_gbq('hello.2019DataSecondhalf', 
                 Context.default().project_id,
                 chunksize=500000, 
                 if_exists='append',
                 verbose=False
                 )

/usr/local/lib/python3.6/dist-packages/pandas_gbq/gbq.py:1127: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  stacklevel=1,
3it [00:37, 10.32s/it]


<b> <u> Taxi Zone Data Visulizations </u></b>

In [0]:
query = """
WITH ZONE AS (
SELECT *,ST_CENTROID(zone_geom) as p FROM `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`
)
SELECT * ,ST_X(p) as longitude,
  ST_Y(p) as latitude FROM ZONE

"""
df = client.query(query).to_dataframe()

In [0]:
import dask.dataframe as dd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import folium
from datetime import datetime
import time
import seaborn as sns
import os
import math
import warnings

In [0]:
#detecting the pickups latitude and longitudes which are outside NYC.
outside_NYC = df[(df.latitude != 0) | (df.longitude != 0) ]
m = folium.Map(location = [40.5774, -73.7004], tiles = "Stamen Toner")
outside_pickups = outside_NYC.head(25000)
for i,j in outside_pickups.iterrows():
    if j["latitude"] != 0:
        folium.Marker(location=[j["latitude"], j["longitude"]],popup=j["zone_name"]).add_to(m)
m

In [0]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/drive/My Drive/Data Science/nyc-taxi-265120-baf9a3e4cf9b.json"
%reload_ext google.cloud.bigquery

In [0]:
project_id = 'nyc-taxi-265120' 
from google.cloud import bigquery
client = bigquery.Client(project = project_id)

In [0]:
query = """
SELECT  * FROM `hello.2019DataFirsthalf`
"""
first_2019 = client.query(query).to_dataframe()

In [0]:
query = """
SELECT  * FROM `hello.2019DataSecondhalf`
"""
second_2019 = client.query(query).to_dataframe()

In [0]:

query = """
SELECT  * FROM `hello.2018DataSecondhalf`
"""
second_2018 = client.query(query).to_dataframe()

In [0]:
query = """
SELECT  * FROM `hello.2018DataFirsthalf`
"""
first_2018 = client.query(query).to_dataframe()

In [0]:
import pandas as pd
concat=pd.concat([first_2018,second_2018,second_2019,first_2019],ignore_index=True)

In [0]:
!pip install datalab

In [0]:
from datalab.context import Context
#Dont run this

concat.to_gbq('hello.CombinedData', 
                 Context.default().project_id,
                 chunksize=500000, 
                 if_exists='append',
                 verbose=False
                 )

/usr/local/lib/python3.6/dist-packages/pandas_gbq/gbq.py:1127: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  stacklevel=1,
10it [02:25, 14.71s/it]


<p align="justify"> <b>Finally Data is prepared in a format that can be modelled and explored easily. Also the outliers from Data has been removed. </b>
</p>